In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# TELCO CHURN PROJECT

The purpose is to see if I can predict which customers will stay and which will leave. 

I will use sklearn to run the regression and determine the important factors.

In [ ]:
telco = pd.read_csv('/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [ ]:
telco.tail()

# lets take a look

In [ ]:
filt = telco['Churn'] == 'Yes'
gone = telco[filt]
stayed = telco[~filt]

In [ ]:
gone.tail()

In [ ]:
stayed.tail()

so I can see the columns. I will remove the ID because it is useless.

I will have to get dummies for some of the variables when I do the regression at the end.

But I will first take a look at some of the numerical categories.
I have created data sets for churned and unchurned customers to visualise the difference if neccessary. 

In [ ]:
telco.dtypes

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from pylab import rcParams
rcParams['figure.figsize']=20,10

In [ ]:
# I will look at tenure. The no. of months the customer has been with the company.


x = telco['tenure']
y = gone['tenure']
z = stayed['tenure']

fig,((ax1, ax2, ax3)) = plt.subplots(1,3)

ax1.hist(x)
ax1.set_title('Telco')

ax2.hist(y)
ax2.set_title('Gone')

ax3.hist(z)
ax3.set_title('Stayed')

So the majority of people have only been there a short amount of time, most under 10 months.

Of the leavers, this number is bigger, whereas people who have been there a long time are less likely to quit.

This was largely to be expected




In [ ]:
# I will look at monthly charges.

x = telco['MonthlyCharges']
y = gone['MonthlyCharges']
z = stayed['MonthlyCharges']

fig,((ax1, ax2, ax3)) = plt.subplots(1,3)

ax1.hist(x)
ax1.set_title('Telco')

ax2.hist(y)
ax2.set_title('Gone')

ax3.hist(z)
ax3.set_title('Stayed')

Interestingly, people who left were spending more money.
People who stayed were mostly spending smaller amounts.

In [ ]:
telco['TotalCharges'] = pd.to_numeric(telco['TotalCharges'], errors='coerce')
gone['TotalCharges'] = pd.to_numeric(gone['TotalCharges'], errors='coerce')
stayed['TotalCharges'] = pd.to_numeric(stayed['TotalCharges'], errors='coerce')

# this is here because it was taking too long

In [ ]:
# I will look at monthly charges.

rcParams['figure.figsize']= 10,20

x = telco['TotalCharges']
y = gone['TotalCharges']
z = stayed['TotalCharges']


In [ ]:
fig,((ax1, ax2, ax3)) = plt.subplots(3,1)

ax1.hist(x)
ax1.set_title('Telco')
ax1.set_xticks(np.arange(0,7000,1000))

ax2.hist(y)
ax2.set_title('Gone')
ax2.set_xticks(np.arange(0,3000,1000))

ax3.hist(z)
ax3.set_title('Stayed')
ax3.set_xticks(np.arange(0,7000,1000))

This graph isn't too informative.





# Feature Engineering

I am going to do some feature engineering with the numerical categories.

In [ ]:
# I'll make a new column with five cateogies for Monthly charges.
mc = telco['MonthlyCharges']

In [ ]:
print(max(mc), min(mc))

In [ ]:
telco['Monthly'] = 'under40'

In [ ]:
telco['MonthlyCharges']

In [ ]:
filt=telco['MonthlyCharges']>40
telco.loc[filt, 'Monthly']='20-40'


In [ ]:
filt=telco['MonthlyCharges']>60
telco.loc[filt, 'Monthly']='40-60'

In [ ]:
filt=telco['MonthlyCharges']>80
telco.loc[filt, 'Monthly']='80-100'

In [ ]:
filt=telco['MonthlyCharges']>100
telco.loc[filt, 'Monthly']='over 100'

In [ ]:
telco['Monthly']

In [ ]:
# I'll make a new column with five cateogies for Monthly charges.
t = telco['tenure']

In [ ]:
max(t)

In [ ]:
telco['New_tenure'] = 'Long-standing'

In [ ]:
filt=telco['tenure']<24
telco.loc[filt, 'New_tenure']='1-2 Years'

In [ ]:
filt=telco['tenure']<12
telco.loc[filt, 'New_tenure']='1st year'

In [ ]:
tc= telco['TotalCharges']

In [ ]:
telco['total'] = 'big_spenders'

In [ ]:
filt=telco['TotalCharges']<5000
telco.loc[filt, 'total']='regulars'

In [ ]:
filt=telco['TotalCharges']<1000
telco.loc[filt, 'total']='newbies'

In [ ]:
telco['total'].value_counts()

In [ ]:
telco['New_tenure'].value_counts()

In [ ]:
telco['Monthly'].value_counts()

these have no been broken down into more useful categories.

In [ ]:
telco.columns

# Further exploratory analysis.

a further look at the categorical variables

In [ ]:
x = telco['gender']
y = gone['gender']
z = stayed['gender']

rcParams['figure.figsize']= 20,5

fig,((ax1, ax2, ax3)) = plt.subplots(1,3)

ax1.hist(x)
ax1.set_title('Telco')

ax2.hist(y)
ax2.set_title('Gone')

ax3.hist(z)
ax3.set_title('Stayed')

Nothing too interesting here.

In [ ]:
x = telco['SeniorCitizen']
y = gone['SeniorCitizen']
z = stayed['SeniorCitizen']

rcParams['figure.figsize']= 20,5

fig,((ax1, ax2, ax3)) = plt.subplots(1,3)

ax1.hist(x)
ax1.set_title('Telco')

ax2.hist(y)
ax2.set_title('Gone')

ax3.hist(z)
ax3.set_title('Stayed')

this appears to be relevant

In [ ]:
x = telco['Partner']
y = gone['Partner']
z = stayed['Partner']

rcParams['figure.figsize']= 20,5

fig,((ax1, ax2, ax3)) = plt.subplots(1,3)

ax1.hist(x)
ax1.set_title('Telco')

ax2.hist(y)
ax2.set_title('Gone')

ax3.hist(z)
ax3.set_title('Stayed')

ok this seems to be relevant also.

In [ ]:
telco['PaymentMethod'].value_counts()

In [ ]:
x = telco['PaymentMethod']
y = gone['PaymentMethod']
z = stayed['PaymentMethod']

rcParams['figure.figsize']= 10,20

fig,((ax1, ax2, ax3)) = plt.subplots(3,1)

ax1.hist(x)
ax1.set_title('Telco')

ax2.hist(y)
ax2.set_title('Gone')

ax3.hist(z)
ax3.set_title('Stayed')

People with Electronic check appear to be leaving more than others. Potentially significant.

# preparing the regression



In [ ]:
telco.head()

In [ ]:
pd.set_option('display.max_columns', 50)
telco.head()

In [ ]:
telco.drop(['customerID','tenure','MonthlyCharges','TotalCharges'], axis=1, inplace=True)

In [ ]:
telco = pd.get_dummies(telco,drop_first=False)

telco.drop('Churn_No', axis=1, inplace=True)
telco.head()

# ready to go

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(telco.drop('Churn_Yes', axis=1), telco['Churn_Yes'])

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LogReg = LogisticRegression()
LogReg.fit(x_train, y_train)

In [ ]:
LogReg.score(x_train, y_train)

In [ ]:
LogReg.score(x_test, y_test)

In [ ]:
LogReg_summary = pd.DataFrame(x_train.columns.values, columns=['Features'])

In [ ]:
coefs = LogReg.coef_
coefs.shape

In [ ]:
coefs = coefs.reshape(53,1)

In [ ]:
LogReg_summary['coefs'] = coefs

In [ ]:
from sklearn.feature_selection import f_regression

In [ ]:
f_regression(x_train, y_train)
p_values = f_regression(x_train, y_train)[1]
LogReg_summary['p_value'] = p_values.round(4)
LogReg_summary.sort_values('coefs')

# done

here we can see the breakdown.

The Length of contract seems to be the most important factor. It is better to push people towards longer contracts.

Also, new clients are much more likely to churn than old clients.

Type of internet service appears to be important also with fibre optics having larger churn and DSL (older) correlating with less churn



